# Notebook 1: Text completion with Oobabooga

> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

**Step 1**: Configure your Oobabooga service settings

You need to have an Oobabooga instance running with API enabled with blocking and streaming ports available at an endpoint.
Use [this notebook](0-AI-settings.ipynb) to save your Oobabooga settings in the configuration file.

We're retrieving the necessary configurations. This step is crucial for the rest of the notebook.

In [6]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs

//Import package for loading hierarchical settings from settings.json
#r "nuget: Microsoft.Extensions.Configuration"
#r "nuget: Microsoft.Extensions.Configuration.Json"
#r "nuget: Microsoft.Extensions.Configuration.Binder"

//Import multiconnector package for loading hierarchichal settings from settings.json, discarding the previous import

#r "nuget: MyIA.SemanticKernel.Connectors.AI.MultiConnector"

using MyIA.SemanticKernel.Connectors.AI.MultiConnector.Configuration;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load configuration using builder package

var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("config/settings.json", optional: false, reloadOnChange: true);

IConfiguration configuration = builder.Build();

 var oobaboogaConfiguration = configuration.GetSection("Oobabooga").Get<OobaboogaConnectorConfiguration>();

Installed Packages Microsoft.Extensions.Configuration, 7.0.0 Microsoft.Extensions.Configuration.Binder, 7.0.4 Microsoft.Extensions.Configuration.Json, 7.0.0 MyIA.SemanticKernel.Connectors.AI.MultiConnector, 0.34.0

**Step 2**: Import Semantic Kernel SDK and Oobabooga from NuGet

We're importing the required packages. Semantic Kernel is the core, and Oobabooga is the connector for text completion.

In [7]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel"
// Import Oobabooga connector
#r "nuget: MyIA.SemanticKernel.Connectors.AI.Oobabooga"

Installed Packages microsoft.semantickernel, 1.0.0-beta2 myia.semantickernel.connectors.ai.oobabooga, 0.34.0

**Step 3**: Create Oobabooga Text completion settings

We're setting up the text completion settings, specifying endpoints and ports. Many other parameters are available to control concurrency and oobabooga's generation parameters.

In [8]:
using System;
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Reflection;
using MyIA.SemanticKernel.Connectors.AI.Oobabooga.Completion.TextCompletion;

// Build settings from configuration file
var oobaboogaTextCompletionSettings = new OobaboogaTextCompletionSettings(
    endpoint: new Uri(oobaboogaConfiguration.EndPoint),
    blockingPort: oobaboogaConfiguration.BlockingPort,
    streamingPort: oobaboogaConfiguration.StreamingPort
);

// Serialize to JSON
string jsonString = JsonSerializer.Serialize(oobaboogaTextCompletionSettings, new JsonSerializerOptions { WriteIndented = true });

// Display the JSON string
Console.WriteLine($"Serialized settings: {jsonString}");


Serialized settings: {
  "OverrideRequestSettings": false,
  "OobaboogaParameters": {
    "max_new_tokens": null,
    "auto_max_new_tokens": null,
    "preset": "None",
    "do_sample": true,
    "temperature": 0,
    "top_p": 0,
    "typical_p": 1,
    "epsilon_cutoff": 0,
    "eta_cutoff": 0,
    "tfs": 1,
    "top_a": 0,
    "repetition_penalty": 1.18,
    "top_k": 20,
    "min_length": 0,
    "no_repeat_ngram_size": 0,
    "num_beams": 1,
    "penalty_alpha": 0,
    "length_penalty": 1,
    "early_stopping": false,
    "mirostat_mode": 0,
    "mirostat_tau": 5,
    "mirostat_eta": 0.1,
    "guidance_scale": 1,
    "negative_prompt": "",
    "seed": -1,
    "add_bos_token": true,
    "truncation_length": 2048,
    "ban_eos_token": false,
    "skip_special_tokens": true,
    "stopping_strings": [],
    "chat_system_prompt": "Assistant is a large language model.",
    "service_id": null,
    "model_id": null
  },
  "WebSocketBufferSize": 2048,
  "BlockingUri": "http://192.168.0.39:500

**Step 4**: Basic Text Completion

We're doing a basic text completion. You'll see how the model currently hosted by default on your Oobabooga endpoint completes a given text prompt.

In [11]:
using Microsoft.SemanticKernel.AI.TextCompletion;
using MyIA.SemanticKernel.Connectors.AI.Oobabooga.Completion;
 
var inputPrompt = "Once upon a time, ";

var oobaboogaLocal = new OobaboogaTextCompletion(oobaboogaTextCompletionSettings);

 var localResponse = await oobaboogaLocal.CompleteAsync(inputPrompt, new OobaboogaCompletionRequestSettings()
 {
     Temperature = 0.01,
     MaxNewTokens = 20,
     TopP = 0.1,
 });

 Console.WriteLine(localResponse);

1980 to be exact, there was an idea. A group of people got together and


**Step 5**: Streaming Text Completion

We're taking it up a notch with streaming text completion. This allows for more dynamic and real-time completions.

In [13]:
using Microsoft.SemanticKernel.AI.TextCompletion;
using MyIA.SemanticKernel.Connectors.AI.Oobabooga.Completion;
 
var inputPrompt = "Once upon a time, ";

var oobaboogaLocal = new OobaboogaTextCompletion(oobaboogaTextCompletionSettings);


var localResponse = oobaboogaLocal.CompleteStreamAsync(inputPrompt,  new OobaboogaCompletionRequestSettings()
{
    Temperature = 0.01,
    MaxNewTokens = 20,
    TopP = 0.1,
});

await foreach (var result in localResponse)
{
    // This will output each piece of the result as it comes in
    Console.Write(result);
}


1980 to be exact, there was an idea. A group of people got together and

**Step 6**: Use Kernel with Oobabooga

We're integrating Oobabooga with a Semantic Kernel's kernel. This allows for more advanced functionalities.

In [14]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.TextCompletion;
using MyIA.SemanticKernel.Connectors.AI.Oobabooga;


//Create Kernel builder
var builder = new KernelBuilder();

builder.WithOobaboogaTextCompletionService(oobaboogaTextCompletionSettings);

IKernel kernel = builder.Build();

// Get text completion service from kernel
var textCompletion = kernel.GetService<ITextCompletion>();
var response = await textCompletion.CompleteAsync("Hello, my name is", new OobaboogaCompletionRequestSettings(){
     Temperature = 0.01,
     MaxNewTokens = 20,
     TopP = 0.1,
 });
Console.WriteLine(response);

 Katie and I am a 20 year old college student. I have been diagnosed with


**Step 7**: Load and Run a Skill

We're loading a skill and running it. Skills are pre-defined functions that can be invoked to perform specific tasks.

In [15]:
using Microsoft.SemanticKernel.Orchestration;
// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "samples", "skills");

// Load the FunSkill from the Skills Directory
var funSkillFunctions = kernel.ImportSemanticFunctionsFromDirectory(skillsDirectory, "FunSkill");

 var contextVariables = new ContextVariables("time travel to dinosaur age");
 contextVariables["style"] = "English humor";
 
 var context = kernel.CreateNewContext(contextVariables);

// Run the Function called Joke
var result = await funSkillFunctions["Joke"].InvokeAsync(context);

// Return the result to the Notebook
Console.WriteLine(result);


Joke: Why did the time-traveling scientist go back in time? To find out why they went extinct!


**Step 8**: Advanced Text Completion with Custom Chat History

In this section, we'll show you how to simulate a chat experience by maintaining a custom chat history. We'll use text completion to generate responses.
This is a sneak peek into more advanced capabilities.

In [18]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.Orchestration;

// Initialize Kernel Builder
var builder = new KernelBuilder();

// Configure Oobabooga as the AI backend
builder.WithOobaboogaTextCompletionService(oobaboogaTextCompletionSettings);

// Build the Kernel
IKernel kernel = builder.Build();

// Define the prompt
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

// Configure the prompt

 var promptTemplateConfig = new PromptTemplateConfig();
 promptTemplateConfig.ModelSettings.Add(new OobaboogaCompletionRequestSettings()
{
    Temperature = 0.7,
    MaxNewTokens = 100,
});


// Register the semantic function
var promptTemplate = new PromptTemplate(skPrompt, promptTemplateConfig, kernel);
var chatFunction = kernel.RegisterSemanticFunction("ChatBot", "Chat", promptTemplateConfig, promptTemplate);

// Initialize context
var context = kernel.CreateNewContext();
var history = "";
context.Variables["history"] = history;

// Chat function
Func<string, Task> Chat = async (string input) => {
    context.Variables["userInput"] = input;
    var answer = await chatFunction.InvokeAsync(context);
    history += $"\nUser: {input}\nChatBot: {answer}\n";
    context.Variables["history"] = history;
    Console.WriteLine($"ChatBot: {answer}");
};

// Example usage
var message = "Tell me a joke.";
Console.WriteLine($"User: {message}");
await Chat(message);
message = "Why is that funny?";
Console.WriteLine($"User: {message}");
await Chat(message);

User: Tell me a joke.
ChatBot:  Why did the tomato turn red? Because it saw the salad dressing!
User: Why is that funny?
ChatBot:  It plays on the idea of why something turns red, and adds a humorous twist by saying it's because it saw the salad dressing.


You've got the basics down. Ready to dive deeper? Head over to Notebook 2 for more advanced chat capabilities.

**Next Steps**: You know the basics, you can now move on to chat capabilities.